In [51]:
import numpy as np
import pandas as pd 
import os
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold,StratifiedKFold,RepeatedStratifiedKFold
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [52]:
train_df      = pd.read_csv('archive/train.csv')
test_df       = pd.read_csv('archive/test.csv')
submission_df = pd.read_csv('archive/sample_submission.csv')

In [54]:
#Creating Addition Features
combine_set=pd.concat([train_df,test_df],axis=0)
combine_set['City_Code_Patient'].fillna(-99,inplace=True)
combine_set['Bed Grade'].fillna(5,inplace=True)
combine_set['Unique_Hospital_per_patient']=combine_set.groupby(['patientid'])['Hospital_code'].transform('nunique')
combine_set['Unique_patient_per_hospital']=combine_set.groupby(['Hospital_code'])['patientid'].transform('nunique')
combine_set['Unique_patient_per_Department']=combine_set.groupby(['Department'])['patientid'].transform('nunique')
combine_set['Total_deposit_paid_by_patient_in_each_hospital']=combine_set.groupby(['Hospital_code','patientid'])['Admission_Deposit'].transform('sum')
combine_set['Min_Severity_of_Illness'] = combine_set.groupby('patientid')['Severity of Illness'].transform('min')

In [55]:

le = LabelEncoder()
#Encoding categorical variables by frequency encoding and label encoding
for col in combine_set.select_dtypes(include='object').columns:
    if col not in ['Age','Stay']:
        fe=combine_set.groupby([col]).size()/len(combine_set)
        combine_set[col]=combine_set[col].apply(lambda x: fe[x])   
        # combine_set[col]  = pd.get_dummies(combine_set[col].astype(str))         
    elif col!='Stay':
        combine_set[col]=le.fit_transform(combine_set[col].astype(str))
    else:
        pass


In [56]:
#Splitting train and test

X=combine_set[combine_set['Stay'].isnull()==False].drop(['case_id','Stay'],axis=1)
y=le.fit_transform(combine_set[combine_set['Stay'].isnull()==False]['Stay'])
y=pd.DataFrame(y,columns=['Stay'])
X_main_test=combine_set[combine_set['Stay'].isnull()==True].drop(['case_id','Stay'],axis=1)

In [57]:
# kf=KFold(n_splits=10,shuffle=True,random_state=294)
kf=KFold(n_splits=5,shuffle=True)

preds_1   = {}
y_pred_1  = []
acc_score = []

for i,(train_idx,val_idx) in enumerate(kf.split(X)):    
    
    X_train, y_train = X.iloc[train_idx,:], y.iloc[train_idx]
    
    X_val, y_val = X.iloc[val_idx, :], y.iloc[val_idx]

    print('\nFold: {}\n'.format(i+1))




Fold: 1


Fold: 2


Fold: 3


Fold: 4


Fold: 5



In [58]:

num_classes = len(np.unique(y_train))
print("Nombre de classes réelles :", num_classes)

Nombre de classes réelles : 11


In [59]:


# Vérifier le type de y_train
print("Type de y_train :", type(y_train))

# Vérifier les valeurs uniques dans y_train
unique_classes = np.unique(y_train)
print("Classes réelles :", unique_classes)
print("Nombre de classes réelles :", len(unique_classes))

Type de y_train : <class 'pandas.core.frame.DataFrame'>
Classes réelles : [ 0  1  2  3  4  5  6  7  8  9 10]
Nombre de classes réelles : 11


In [18]:
lg=LGBMClassifier(device="gpu", 
                      gpu_platform_id= 0,
                      max_bin=63,#Theoretically best speeds using LGBM
                      gpu_device_id= 0,
                      boosting_type='gbdt',
                      learning_rate=0.04,
                      # max_depth=15,
                      # num_leaves = 150,
                      objective='multi_class',
                      num_class=11,                      
                      n_estimators=50000,
                      metric='multi_error',
                      colsample_bytree=0.8,
                      min_child_samples=228,
                      reg_alpha=1,
                      reg_lambda=1,
                      # random_state=294,
                      n_jobs=-1,

                       ) 

In [ ]:
 
# cette entrainement ne marche pas  ma il me faut update mon Classifier

lg.fit(X_train, y_train
    #,categorical_feature = categorical_features
    ,eval_metric='multi_error'
    ,eval_set=[(X_train, y_train),(X_val, y_val)]
    ,early_stopping_rounds=100
    ,verbose=50
)




In [61]:
# Marche
import lightgbm as lgb

# Préparation des données d'entraînement X_train et y_train

#  les paramètres du modèle
#diminuer le nombre de parametre n_estimators sa peut cracher ma machine
params = {
    'max_bin':63,
    'boosting_type':'gbdt',
    'learning_rate':0.04,
    'objective': 'multiclass',
    'metric': ['multi_logloss'],
    'num_class': 11,
    'n_estimators':5000,
    'metric':'multi_error',
    'colsample_bytree':0.8,
    'min_child_samples':228,
    'reg_alpha':1,
    'reg_lambda':1,
    'n_jobs':-1,
      
    # Autres paramètres du modèle
}

# Création d'un dataset LightGBM à partir des données d'entraînement
train_data = lgb.Dataset(X_train, label=y_train)

# Entraînement du modèle
model = lgb.train(params, train_data)

# Utilisation le modèle pour faire des prédictions donnee de test

predictions = model.predict(X_val)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016402 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 430
[LightGBM] [Info] Number of data points in the train set: 254751, number of used features: 21
[LightGBM] [Info] Start training from score -2.602660
[LightGBM] [Info] Start training from score -1.405504
[LightGBM] [Info] Start training from score -1.291934
[LightGBM] [Info] Start training from score -1.753033
[LightGBM] [Info] Start training from score -3.304269
[LightGBM] [Info] Start training from score -2.207904
[LightGBM] [Info] Start training from score -4.747294
[LightGBM] [Info] Start training from score -3.432622
[LightGBM] [Info] Start training from score -4.177517
[LightGBM] [Info] Start training from score -4.736493
[LightGBM] [Info] Start tra

KeyboardInterrupt: 

In [38]:
#affichage des predictions
print(predictions)

[[0.04423404 0.14202437 0.46353668 ... 0.00295867 0.0018761  0.03028334]
 [0.05435923 0.17139041 0.42558638 ... 0.00465259 0.00416769 0.03070038]
 [0.05511342 0.38970614 0.33962049 ... 0.00454809 0.0030121  0.00792457]
 ...
 [0.0083145  0.08465096 0.01702956 ... 0.00352462 0.01987457 0.01629955]
 [0.12011986 0.32937458 0.34355589 ... 0.00130039 0.00197271 0.00142066]
 [0.12672843 0.40300733 0.28907694 ... 0.00212159 0.00151663 0.00264064]]


In [41]:
from sklearn.metrics import classification_report

# Convertir les prédictions en classes
y_pred_class = [np.argmax(pred) for pred in predictions]

# Calculer le rapport de classification
report = classification_report(y_val, y_pred_class)

# Afficher le rapport de classification
print(report)

              precision    recall  f1-score   support

           0       0.43      0.16      0.23      4745
           1       0.43      0.50      0.46     15598
           2       0.42      0.67      0.52     17401
           3       0.44      0.23      0.30     11074
           4       0.00      0.00      0.00      2324
           5       0.39      0.51      0.45      7025
           6       0.20      0.00      0.00       581
           7       0.31      0.01      0.02      2111
           8       0.36      0.15      0.21       947
           9       0.49      0.03      0.06       542
          10       0.55      0.43      0.48      1339

    accuracy                           0.43     63687
   macro avg       0.37      0.24      0.25     63687
weighted avg       0.41      0.43      0.39     63687



In [ ]:
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, precision_recall_curve, auc


# Calculer les courbes ROC ne se trace pas pour les probleme de classification multiclasse uniquement binaire mais on peut faire plusieurs courbe

# fpr, tpr, _ = roc_curve(X_val, y_val)
# roc_auc = auc(fpr, tpr)

# Tracer la courbe ROC

# plt.figure()
# plt.plot(fpr, tpr, color='darkorange', lw=2, label='Courbe ROC (AUC = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('Taux de faux positifs')
# plt.ylabel('Taux de vrais positifs')
# plt.title('Courbe ROC')
# plt.legend(loc="lower right")
# plt.show()

# Calculer les courbes de précision-rappel

precision, recall, _ = precision_recall_curve(X_val, y_val)

# Tracer la courbe de précision-rappel
plt.figure()
plt.plot(recall, precision, color='blue', lw=11, label='Courbe de précision-rappel')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Rappel')
plt.ylabel('Précision')
plt.title('Courbe de précision-rappel')
plt.legend(loc="lower right")
plt.show()

In [ ]:
print(accuracy_score(y_val,lg.predict(X_val))*100)
    
acc = accuracy_score(y_val,lg.predict(X_val))*100
acc_score.append(acc)
print("Score : ",acc)    
y_pred_1.append(lg.predict_proba(X_main_test))
    


# preds_1[i+1]=lg.predict_proba(X_main_test)
# y_pred_1.append(lg.predict_proba(X_main_test))


In [ ]:
y_pred_final_1 = np.mean(np.array(y_pred_1),axis=0)
    
print('mean accuracy score: {}'.format((sum(acc_score)/10)))

preds_1=np.argmax(y_pred_final_1,axis=1)

print(preds_1.shape)
submission_df['Stay']=le.inverse_transform(preds_1.astype(int))
# submission_df[0] = y_pred_final_1[:,0]
# submission_df[1] =y_pred_final_1[:,1]

# Download Submission File :
display("submission_df",submission_df)
sub_file_name_1 = "BEST_11_CV=42.96_LB=WAIT_LGBM-1.csv"


submission_df.to_csv(sub_file_name_1,index=False)
submission_df.head(5)